In [ ]:
# Liturgický kalendár:
# v tabuľkovom zobrazení (12 mesiacov) ako "one page".

In [11]:
import os
from bs4 import BeautifulSoup

In [12]:
# kde sú html súbory:
data_dir = './data_html/'

In [13]:
# List all html files from data directory with file name length 12 (meaning particular date like: '2023-12.html')
# poznámka: 
# keďže cirkevný rok začína adventom, sú v zip archíve aj dni z predošlého roka (2022)
# preto som pre názornosť dal podmienku, aby bral len aktuálny rok 2023 (file.startswith('2023'))
html_files = [ file for file in os.listdir(data_dir) if file.startswith('2023') and file.endswith(".html") and len(file)==12 ]

len(html_files) 

12

In [14]:
for i,fname in enumerate(html_files):

    with open(data_dir+fname, encoding='utf-8') as f_in:
        s = f_in.read()            
    
    if i==0: # prvý súbor, vytiahneme HTML kostru
        soup1 = BeautifulSoup(s,'lxml') # 
        soup1.title.string = 'kalendár 2023, tabuľka' # meníme title

        tag = soup1.find('a', class_='lcPREV')
        if tag:
            tag.extract()

        tag = soup1.find('a', class_='lcNEXT')
        if tag:
            tag.extract()

        odkazy = soup1.find_all('a', class_='lcDEN')
        for odk in odkazy:
            odk['href'] = 'onepage.html#'+odk['href'][:-5]

    else: 
        # parsujeme: 
        hr_tag = soup1.new_tag('hr')
        hr_tag['id'] = fname[:-5] # id='2023-01' napríklad
        soup1.html.append(hr_tag) # pridávame hr

        soup2 = BeautifulSoup(s,'lxml') #    

        soup3 = soup2.body
        soup3.script.extract() # init(); preč

        tag = soup3.find('a', class_='lcPREV')
        if tag:
            tag.extract()

        tag = soup3.find('a', class_='lcNEXT')
        if tag:
            tag.extract()

        odkazy = soup3.find_all('a', class_='lcDEN')
        for odk in odkazy:
            odk['href'] = 'onepage.html#'+odk['href'][:-5]
        # soup3.h1.a.replace_with(soup2.h1.a.text) # úplne odstraňujeme link z nadpisu. pre zmenu linku by sa použilo: soup2.h1.a['href']= ''

        soup1.html.append(soup3)

In [15]:
with open('onepage_tabulka2.html', mode='w', encoding='utf-8') as f_out:
    f_out.write(str(soup1))

In [ ]:
# done.